In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

data_train = spi.loadmat('train_PDE_ADVD.mat')

u_in = data_train['X_train0']
x_t_in = data_train['X_train1']
s_in = data_train['y_train']

u_in.shape, x_t_in.shape, s_in.shape

((100000, 100), (100000, 2), (100000, 1))

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = 100000

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

tfd = tfp.distributions
tfb = tfp.bijectors

def normal_sp(params):
    return tfd.Normal(loc = params[:, 0:1], scale = 0.001+tf.math.softplus(params[:, 1:2]))    

def negloglikelihood(y_true, y_pred):
    return tf.keras.backend.sum(-y_pred.log_prob(y_true))+(sum(model.losses)/bs)

hln = 35

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (2,), name = 'inputsT')
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = tfp.layers.DenseFlipout(2)(combined)

dist = tfp.layers.DistributionLambda(normal_sp)(output)
model = Model(inputs = [inputsB, inputsT], outputs = dist)    

model.summary()

C:\Users\shailesh\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_probability\python\layers\util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  loc = add_variable_fn(
C:\Users\shailesh\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_probability\python\layers\util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  untransformed_scale = add_variable_fn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 2)]          0           []                               
                                                                                                  
 dense_flipout (DenseFlipout)   (None, 35)           7035        ['inputsB[0][0]']                
                                                                                                  
 dense_flipout_3 (DenseFlipout)  (None, 35)          175         ['inputsT[0][0]']                
                                                                                              

In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
spe = 25
string = './model/modelC4_S1_/'+str(spe)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss_value = 0
        for i in range(0,spe):        
            logits = model({"inputsB":u_in, "inputsT":x_t_in}, training=True)
            loss_value = loss_value + negloglikelihood(s_in, logits)
        loss_value = loss_value*(1/spe)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

epochs = 10000
loss = np.zeros(epochs)

for epoch in range(epochs):
    loss_value = train_step()
    loss[epoch] = loss_value.numpy()
    if loss[epoch] <= np.min(loss[0:epoch+1]):
        model.save_weights(string)
        last_saved_wt = epoch
    if epoch%10 == 0:
        print("Epoch %d, loss %.2f" % (epoch, loss[epoch]))

print(last_saved_wt)

Epoch 0, loss 98778.48
Epoch 10, loss 96796.16
Epoch 20, loss 92387.11
Epoch 30, loss 88785.96
Epoch 40, loss 86650.29
Epoch 50, loss 84265.85
Epoch 60, loss 82161.11
Epoch 70, loss 79693.80
Epoch 80, loss 78118.35
Epoch 90, loss 76280.92
Epoch 100, loss 74288.95
Epoch 110, loss 72288.89
Epoch 120, loss 70487.96
Epoch 130, loss 68522.03
Epoch 140, loss 69157.39
Epoch 150, loss 64608.23
Epoch 160, loss 62815.16
Epoch 170, loss 61782.79
Epoch 180, loss 59464.08
Epoch 190, loss 58922.18
Epoch 200, loss 54458.47
Epoch 210, loss 53855.80
Epoch 220, loss 53010.52
Epoch 230, loss 50939.17
Epoch 240, loss 51222.13
Epoch 250, loss 50328.01
Epoch 260, loss 48201.49
Epoch 270, loss 48554.84
Epoch 280, loss 45105.72
Epoch 290, loss 43516.95
Epoch 300, loss 43246.69
Epoch 310, loss 41460.08
Epoch 320, loss 40558.35
Epoch 330, loss 41593.46
Epoch 340, loss 39304.34
Epoch 350, loss 37403.43
Epoch 360, loss 37521.10
Epoch 370, loss 34182.15
Epoch 380, loss 34700.66
Epoch 390, loss 30740.71
Epoch 400, 

In [4]:
data_test = spi.loadmat('test_PDE_ADVD.mat')

u_in_test = data_test['X_test0']
x_t_in_test = data_test['X_test1']
s_in_test = data_test['y_test']

nsamples = 100
nps = 1
pred = np.zeros([nsamples,1000000])
for i in range(0,nsamples):
    if i%5 == 0:
        print(i)
    pred[i,:] = np.squeeze((model({"inputsB":u_in_test, "inputsT":x_t_in_test})).sample(1))
    
print()
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2)/np.mean((s_in_test)**2))    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95

0.0018931915236530012
0.41757784508986057
0.004533745134983385


In [5]:
model.save_weights('./model/modelC4stat1')